<div style="text-align: center; font-weight: bold;">
    <h1>Pipeline for Research ready EHR Datasets</h1>
    <h2>Part 3: Aggregating the EHR Data for Analysis</h2>
    <h4>Author: Vidul Ayakulangara Panickan</h3>
</div>




## Step 7 Aggregating Data to Patient level

The level of aggregation depends on your analysis needs. You can do a simple aggregation to 
get EHR code counts per patient as shown below

In [3]:
import pandas as pd
import os

os.makedirs("Aggregated_Data", exist_ok=True)

diagnoses = pd.read_csv("./Rolledup_Data/diagnoses_phecode_rolled.csv",dtype=str)
diagnoses

,subject_id,PheCode,date
0,10000032,571.81,2180-05-06
1,10000032,572,2180-05-06
2,10000032,571.51,2180-05-06
3,10000032,070.3,2180-05-06
4,10000032,496,2180-05-06
...,...,...,...
5922991,19999987,411.4,2145-11-02
5922992,19999987,345.3,2145-11-02
5922993,19999987,041,2145-11-02
5922994,19999987,368.4,2145-11-02


In [4]:
patient_phecode_counts=diagnoses.groupby(['subject_id', 'PheCode']).size().reset_index(name='count')
patient_phecode_counts.to_csv("./Aggregated_Data/patient_phecode_counts.csv",index=None)
patient_phecode_counts

,subject_id,PheCode,count
0,10000032,070.3,4
1,10000032,071,3
2,10000032,260.1,1
3,10000032,276.12,3
4,10000032,276.13,2
...,...,...,...
3564825,19999987,368.4,1
3564826,19999987,411.4,1
3564827,19999987,430.2,1
3564828,19999987,585.1,1


In several cases, you might be interested in studying a particular condition say Asthma. First we identify phecodes of interest, here asthma has a phecode of "495". We will extract all patient data with PheCode 495

In [5]:
asthma_diagnoses = diagnoses[diagnoses['PheCode']=='495']
asthma_diagnoses

,subject_id,PheCode,date
679,10001725,495,2110-04-11
823,10001884,495,2130-12-27
831,10001884,495,2130-12-06
895,10001884,495,2131-01-07
924,10001884,495,2125-10-26
...,...,...,...
5922463,19999112,495,2116-10-28
5922473,19999112,495,2115-02-22
5922491,19999112,495,2115-02-11
5922601,19999442,495,2146-12-19


You can perform a simple aggregation operation to get other diagnoses counts for these patients as done in previous cell.
Sometimes you will be interested in getting monthly counts around first diagnoses of asthma for the patient.

In [6]:
os.makedirs("./Aggregated_Data/Asthma_Monthly_Data", exist_ok=True)

asthma_first_diagnoses = asthma_diagnoses.groupby('subject_id')['date'].min().reset_index()
asthma_first_diagnoses.columns = ['subject_id', 'index_date']
asthma_first_diagnoses

,subject_id,index_date
0,10001725,2110-04-11
1,10001884,2125-10-26
2,10002800,2164-02-23
3,10003019,2174-09-16
4,10004296,2168-10-29
...,...,...
20311,19997760,2187-04-21
20312,19997887,2117-04-07
20313,19998350,2128-02-21
20314,19999112,2114-09-01


We have 20316 unique patients with Astham diagnosis. These dates will be the index date which will be referenced to create monthly counts.

In [7]:
asthma_combined_df = pd.merge(asthma_first_diagnoses,diagnoses[['subject_id','date','PheCode']], on="subject_id",how='left')
asthma_combined_df 

,subject_id,index_date,date,PheCode
0,10001725,2110-04-11,2110-04-11,599.2
1,10001725,2110-04-11,2110-04-11,946
2,10001725,2110-04-11,2110-04-11,618.5
3,10001725,2110-04-11,2110-04-11,495
4,10001725,2110-04-11,2110-04-11,530.11
...,...,...,...,...
978710,19999442,2146-12-19,2148-11-19,856
978711,19999442,2146-12-19,2148-11-19,348
978712,19999442,2146-12-19,2148-11-19,342
978713,19999442,2146-12-19,2148-11-19,495


In [8]:
asthma_combined_df['index_date'] = pd.to_datetime(asthma_combined_df['index_date'])
asthma_combined_df['date'] = pd.to_datetime(asthma_combined_df['date'])

asthma_combined_df['days_from_first_diagnosis'] = (asthma_combined_df['date'] - asthma_combined_df['index_date']).dt.days
asthma_combined_df 

,subject_id,index_date,date,PheCode,days_from_first_diagnosis
0,10001725,2110-04-11,2110-04-11,599.2,0
1,10001725,2110-04-11,2110-04-11,946,0
2,10001725,2110-04-11,2110-04-11,618.5,0
3,10001725,2110-04-11,2110-04-11,495,0
4,10001725,2110-04-11,2110-04-11,530.11,0
...,...,...,...,...,...
978710,19999442,2146-12-19,2148-11-19,856,701
978711,19999442,2146-12-19,2148-11-19,348,701
978712,19999442,2146-12-19,2148-11-19,342,701
978713,19999442,2146-12-19,2148-11-19,495,701


In [9]:
import math

def get_month(days):
    if days == 0:
        return 1

    elif days > 0:
        return math.ceil(days / 30)

    elif days < 0:
        return math.floor(days / 30)
        

asthma_combined_df['month']=asthma_combined_df['days_from_first_diagnosis'].map(lambda x: get_month(x))
asthma_combined_df

,subject_id,index_date,date,PheCode,days_from_first_diagnosis,month
0,10001725,2110-04-11,2110-04-11,599.2,0,1
1,10001725,2110-04-11,2110-04-11,946,0,1
2,10001725,2110-04-11,2110-04-11,618.5,0,1
3,10001725,2110-04-11,2110-04-11,495,0,1
4,10001725,2110-04-11,2110-04-11,530.11,0,1
...,...,...,...,...,...,...
978710,19999442,2146-12-19,2148-11-19,856,701,24
978711,19999442,2146-12-19,2148-11-19,348,701,24
978712,19999442,2146-12-19,2148-11-19,342,701,24
978713,19999442,2146-12-19,2148-11-19,495,701,24


Decide how many months back and forward do you need monthly counts for. Let's say if you are interested in  6 months from the index date and last 6 months before the index date. Then you can do the following 

In [18]:
for month in range(-7,7,1):

    print(f"Processing Month:{month}")
    
    asthma_combined_df_month = asthma_combined_df[asthma_combined_df['month']==month][['subject_id','PheCode']]
    
    phecode_counts_per_subject_month = asthma_combined_df_month.groupby(['subject_id', 'PheCode']).size().reset_index(name='counts')
   
    matrix_df = phecode_counts_per_subject_month.pivot_table(index='subject_id', columns='PheCode', values='counts', fill_value=0)
    
    matrix_df.to_csv("./Aggregated_Data/Asthma_Monthly_Data/month_"+str(month)+".csv")
    

Processing Month:-7
Processing Month:-6
Processing Month:-5
Processing Month:-4
Processing Month:-3
Processing Month:-2
Processing Month:-1
Processing Month:0
Processing Month:1
Processing Month:2
Processing Month:3
Processing Month:4
Processing Month:5
Processing Month:6
